In [1]:
import numpy as np
import pandas as pd
import os
import sys

In [2]:
# Setting path to load util functions
from pathlib import Path
parent_dir = Path.cwd().parents[1]
sys.path.append(os.path.abspath(parent_dir))

In [4]:
from utils.inference_helpers import load_data
df = load_data(data_path='../../data/0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a.parquet')

In [6]:
from utils.inference_helpers import filter_and_impute_df
df = filter_and_impute_df(df, non_null_cols_path='../../utils/selected_features/non_null_cols_filter.csv')

c:\Users\vaseekaranv\Desktop\NIBM-MSc\FYP\utils\inference_helpers.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cat_cols] = df[cat_cols].astype('category')


In [8]:
from utils.inference_helpers import encode_df
df = encode_df(df, cat_encoder_path='../../utils/encoders/categorical_encoder.pkl')

In [9]:
df.shape

(13, 91)

In [12]:
from utils.inference_helpers import create_ts_features
ts_df = create_ts_features(df, imp_cols_path='../../utils/selected_features/boruta_features_baseline.csv',
                           ts_feat_imp_path='../../utils/selected_features/ts_features_boruta.csv')

Aggregated DataFrame type: <class 'pandas.core.frame.DataFrame'>


Creating time series features for each feature: 100%|██████████| 47/47 [00:00<00:00, 115.22it/s]

Concatenating 282 feature DataFrames.


In [14]:
from utils.inference_helpers import create_tsfel_features
tsfel_df = create_tsfel_features(df,
                                 imp_cols_path='../../utils/selected_features/boruta_features_baseline.csv',
                                 stat_select_path='../../utils/selected_features/tsfel_stat_features_boruta.csv',
                                 temporal_select_path='../../utils/selected_features/tsfel_temporal_features_boruta.csv')

Input shape: (13, 49), Grouping by: customer_ID, Timestamp: end_of_month
Using provided TSFEL configuration dictionary.
Processing 1 customer groups...
Setting up Joblib with 20 processes (verbose=0)...

Joblib parallel processing finished in 2.35 seconds.
Successfully processed 1 out of 1 groups.
Preparing results for concatenation...
Concatenating reindexed results...
Concatenation complete.
Robust concatenation finished in 0.17 seconds.
Final features DataFrame shape: (1, 2115)


In [16]:
from utils.inference_helpers import combine_dfs
final_df = combine_dfs(df=df, ts_df=ts_df, tsfel_df=tsfel_df, ref_cols_path='../../data/final_features_ml/val_all_features_last_month.parquet')
final_df.shape

(1, 426)